In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import SGD, Adam
from tensorflow.keras.optimizers import SGD
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

Tạo danh sách label

In [ ]:
word_dict = {
    0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X', 24:'Y',25:'Z'
}

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Tải EMNIST Letters từ tensorflow_datasets
(ds_train_full, ds_test), ds_info = tfds.load(
    'emnist/letters',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

Tiền xử lí ảnh để phù hợp mô hình CNN

In [ ]:
# tiền Xử lý ảnh phù hợp CNN
def pre_process(image, label):
    image = tf.transpose(image, perm=[1, 0, 2])        # Xoay ảnh về đúng chiều
    image = tf.cast(image, tf.float32) / 255.0         # Chuẩn hóa pixel về [0,1]
    label = label - 1                                   # Nhãn từ 1-26 → 0-25
    return image, label

Chia tập dữ liệu train/dev/test

In [ ]:
# Áp dụng tiền xử lí dữ liệu để phù hợp với mô hình CNN
ds_train_full = ds_train_full.map(pre_process, num_parallel_calls=tf.data.AUTOTUNE)

# Chi train là 90%, valid là 10%
val_size = int(0.1 * ds_info.splits['train'].num_examples)

ds_val = ds_train_full.take(val_size).batch(128).prefetch(tf.data.AUTOTUNE)
ds_train = ds_train_full.skip(val_size).batch(128).prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(pre_process).batch(128).prefetch(tf.data.AUTOTUNE)

Xây dựng kiến trúc CNN

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),

    # Thay MaxPooling cuối cùng bằng GlobalAveragePooling2D
    tf.keras.layers.GlobalAveragePooling2D(),

    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(26, activation='softmax')
])


Biên dịch mô hình

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Huấn luyện mô hình

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=30,
    callbacks=[early_stop]
)

test_loss, test_acc = model.evaluate(ds_test)
print(f"Độ chính xác trên tập test: {test_acc:.2%}")

model.save("newModel.h5")
